<a href="https://colab.research.google.com/github/Cate-MSRS/Test_repo/blob/main/Lab_class_4_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ชื่อสมาชิกกลุ่ม
1. ณภณ เรืองกิจจานุวัฒน์ 6704800029
2. ธรรมสรณ์ สาครขำ 6704800035
3. จิตติเทพ วสุนันต์ 6704800052

In [1]:
#!/bin/bash
!curl -L -o ./brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  58.1M      0 --:--:-- --:--:-- --:--:-- 87.5M


In [2]:
!unzip /content/brazilian-ecommerce.zip

Archive:  /content/brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


### 🛠️ Task 1: Load & Clean Data

1. อ่านไฟล์ CSV ทั้ง 4 ไฟล์เข้าสู่ Spark DataFrame
2. ตรวจสอบ Schema และจัดการ Type (ถ้าจำเป็น)
3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL

In [3]:
# ติดตั้ง PySpark (สำหรับ Colab)
!pip -q install pyspark

from pyspark.sql import SparkSession
import os

# สร้าง SparkSession — ประตูเข้าสู่ Spark
# 💡 config("spark.ui.port", "4050") กำหนด port เพื่อเลี่ยง port 4040 ที่อาจชนกัน
spark = SparkSession.builder \
    .appName("BigData-Week4-DeepDive") \
    .config("spark.driver.memory", "2g") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

# เข้าถึง SparkContext ที่ถูก wrap อยู่ข้างใน
sc = spark.sparkContext

print(f"✅ Spark Version: {spark.version}")
print(f"📱 App Name: {sc.appName}")
print(f"🖥️ Master: {sc.master}")


✅ Spark Version: 4.0.2
📱 App Name: BigData-Week4-DeepDive
🖥️ Master: local[*]


In [6]:
df_customers = spark.read.csv("olist_customers_dataset.csv", header=True, inferSchema=True)
df_customers.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows


In [7]:
df_geolocation = spark.read.csv("olist_geolocation_dataset.csv", header=True, inferSchema=True)
df_geolocation.show(5)

+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1037| -23.54562128115268|-46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535|-46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469|-46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681|-46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493|-46.64160722329613|       sao paulo|               SP|
+---------------------------+-------------------+------------------+----------------+-----------------+
only showing top 5 rows


In [9]:
df_order_items = spark.read.csv("olist_order_items_dataset.csv", header=True, inferSchema=True)
df_order_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [10]:
df_order_payments = spark.read.csv("olist_order_payments_dataset.csv", header=True, inferSchema=True)
df_order_payments.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows


In [11]:
df_order_reviews = spark.read.csv("olist_order_reviews_dataset.csv", header=True, inferSchema=True)
df_order_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [12]:
df_order = spark.read.csv("olist_orders_dataset.csv", header=True, inferSchema=True)
df_order.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [13]:
df_products = spark.read.csv("olist_products_dataset.csv", header=True, inferSchema=True)
df_products.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [14]:
df_sellers = spark.read.csv("olist_sellers_dataset.csv", header=True, inferSchema=True)
df_sellers.show(5)

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
+--------------------+----------------------+-----------------+------------+
only showing top 5 rows


In [15]:
df_product_category = spark.read.csv("product_category_name_translation.csv", header=True, inferSchema=True)
df_product_category.show(5)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
+---------------------+-----------------------------+
only showing top 5 rows


In [17]:
df_order.printSchema()
df_order_items.printSchema()
df_products.printSchema()
df_customers.printSchema()


root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable 

In [18]:
df_order.createOrReplaceTempView("orders")
df_order_items.createOrReplaceTempView("items")
df_products.createOrReplaceTempView("products")
df_customers.createOrReplaceTempView("customers")

print("✅ Temp Views Created")


✅ Temp Views Created


### 🔗 Task 2: Join Data

จงสร้าง `df_master` โดยการ Join ตารางดังนี้:
1. `orders` JOIN `items` (ON order_id)
2. JOIN `products` (ON product_id)
3. JOIN `customers` (ON customer_id)

> **Tip:** ตรวจสอบจำนวนแถวหลัง Join ว่าเพิ่มขึ้นหรือลดลงผิดปกติหรือไม่


In [20]:
df_master = df_order \
    .join(df_order_items, "order_id") \
    .join(df_products, "product_id") \
    .join(df_customers, "customer_id")

In [21]:
print("Orders:", df_order.count())
print("Items:", df_order_items.count())
print("Master:", df_master.count())


Orders: 99441
Items: 112650
Master: 112650


### 📊 Task 3: Analytics

ตอบคำถามต่อไปนี้ (เลือกใช้ SQL หรือ PySpark ก็ได้):

1. **Top Products:** สินค้าหมวดไหน (`product_category_name`) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
2. **Top Cities:** เมืองไหน (`customer_city`) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
3. **(Optional) Monthly Sales:** ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)


In [25]:
# Top Products: สินค้าหมวดไหน (product_category_name) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
df_master.createOrReplaceTempView("df_master")
spark.sql("""
SELECT
    product_category_name,
    SUM(price) AS total_sales
FROM df_master
GROUP BY product_category_name
ORDER BY total_sales DESC
LIMIT 5
""").show()

+---------------------+------------------+
|product_category_name|       total_sales|
+---------------------+------------------+
|         beleza_saude| 1258681.340000017|
|   relogios_presentes|1205005.6800000127|
|      cama_mesa_banho|1036988.6800000388|
|        esporte_lazer| 988048.9700000194|
| informatica_acess...| 911954.3200000152|
+---------------------+------------------+



In [30]:
"""
5 อันดับสินค้าที่มียอดขายรวม (price) สูงที่สุด
beleza_saude
relogios_presentes
cama_mesa_banho
esporte_lazer
informatica_acess
"""

In [26]:
# Top Cities: เมืองไหน (customer_city) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
spark.sql("""
SELECT
    customer_city,
    COUNT(DISTINCT order_id) AS total_orders
FROM df_master
GROUP BY customer_city
ORDER BY total_orders DESC
LIMIT 5
""").show()


+--------------+------------+
| customer_city|total_orders|
+--------------+------------+
|     sao paulo|       15402|
|rio de janeiro|        6834|
|belo horizonte|        2750|
|      brasilia|        2116|
|      curitiba|        1510|
+--------------+------------+



In [ ]:
"""
5 อันดับเมืองที่มีจำนวนคำสั่งซื้อมากที่สุด
sao paulo
rio de janeiro
belo horizonte
brasilia
curitiba
"""

In [27]:
# (Optional) Monthly Sales: ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)
from pyspark.sql.functions import date_format

df_monthly = df_master.withColumn(
    "month",
    date_format("order_purchase_timestamp", "yyyy-MM")
)

df_monthly.createOrReplaceTempView("monthly")

spark.sql("""
SELECT
    month,
    SUM(price) AS total_sales
FROM monthly
GROUP BY month
ORDER BY month
""").show()


+-------+------------------+
|  month|       total_sales|
+-------+------------------+
|2016-09|            267.36|
|2016-10| 49507.66000000009|
|2016-12|              10.9|
|2017-01|120312.87000000032|
|2017-02| 247303.0199999985|
|2017-03|374344.29999999516|
|2017-04| 359927.2299999956|
|2017-05| 506071.1399999948|
|2017-06| 433038.5999999939|
|2017-07| 498031.4799999967|
|2017-08| 573971.6799999981|
|2017-09| 624401.6899999977|
|2017-10| 664219.4299999973|
|2017-11|1010271.3700000155|
|2017-12| 743914.1700000004|
|2018-01|  950030.360000015|
|2018-02| 844178.7100000086|
|2018-03| 983213.4400000148|
|2018-04| 996647.7500000133|
|2018-05| 996517.6800000133|
+-------+------------------+
only showing top 20 rows


ยอดขายมีแนวโน้มเติบโตอย่างต่อเนื่องและก้าวกระโดด  โดยเริ่มขยายตัวชัดเจนตั้งแต่ต้นปี 2017 และพุ่งสูงที่สุดในเดือนพฤศจิกายน 2017 (เกิน 1 ล้าน) ก่อนจะเริ่มทรงตัวในระดับสูงอย่างเสถียรในช่วงต้นปี 2018